In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
 

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
train_data = datasets.CIFAR10(root='data',train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='data', train=False, download=True, transform=transform)
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Linear(32*8*8, 10)
        
    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 32*8*8) # x.view(x.size(0),-1)
        return self.fc(x)

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_f = nn.CrossEntropyLoss()


cuda


In [ ]:
epochs = 10
for epoch in range(epochs):
    for batch_idx, (X, Yd) in enumerate(train_loader):
        optimizer.zero_grad()
        X, Yd = X.to(device), Yd.to(device)
        Y = model(X)
        loss = loss_f(Y, Yd)
        loss.backward()
        optimizer.step()
        print(X.shape, Yd.shape, Y.shape)
        print(loss.item())
        break
    with torch.no_grad():
        for X, Yd in test_loader:
            X, Yd = X.to(device), Yd.to(device)
            Y = model(X)
            print(X.shape, Yd.shape, Y.shape)
            break
    break
            

torch.Size([64, 3, 32, 32]) torch.Size([64]) torch.Size([64, 10])
2.32195782661438
torch.Size([64, 3, 32, 32]) torch.Size([64]) torch.Size([64, 10])
